# Inventory Database Migration & Harmonization

**Purpose**
- Migrate `inventory` table schema
- Rename columns:
  - `inventarnummer` → `kv_id`
  - `produkttyp` → `produkt`
- Add new column:
  - `elo_nummer`
- Copy `users` table unchanged
- Write results into a **new database file**

⚠️ This notebook does **NOT** modify the source database in-place.


In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

In [2]:
# ---- PATH CONFIG ----
OLD_DB = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\database_handling\inventory_old.db")
NEW_DB = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\inventory.db")
TARGET_DB = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\database_handling\harmonized_28012026.db")


In [4]:
## Inspect Source Database
src_conn = sqlite3.connect(OLD_DB)

tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    src_conn
)
tables


,name
0,products
1,sqlite_sequence
2,options


In [5]:
pd.read_sql("PRAGMA table_info(products);", src_conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,code,TEXT,1,None,0
2,2,gemeinde,TEXT,1,None,0
3,3,einsatzort,TEXT,0,None,0
4,4,kategorie,TEXT,0,None,0
5,5,produkttyp,TEXT,0,None,0
6,6,produktdetails,TEXT,0,None,0
7,7,anzahl,INTEGER,0,1,0
8,8,hersteller,TEXT,0,None,0
9,9,lieferant,TEXT,0,None,0


In [6]:
## Preview Inventory Data (first 5 rows)
inventory_df = pd.read_sql("SELECT * FROM products LIMIT 5;", src_conn)
inventory_df


,id,code,gemeinde,einsatzort,kategorie,produkttyp,produktdetails,anzahl,hersteller,lieferant,shop_link,preis_netto,preis_brutto,bezahlt,bestellt_am,geliefert_am,uebergeben_am,projekt,bemerkungen
0,1,LL-SJU-2511-0001,St. Julian,innen,EDV & Technik,Digitales Flipchart,,1,,office-partner.de,https://www.office-partner.de/samsung-flip-pro...,1066.39,1269.00,,,,,,
1,2,LL-SJU-2511-0002,St. Julian,innen,EDV & Technik,Beamer,Beamer,1,,Media Markt,https://www.mediamarkt.de/de/product/_epson-eh...,,1845.00,,,,,,
2,3,LL-SJU-2511-0003,St. Julian,innen,EDV & Technik,Mobiler Bildschirm,Smart TV mit Ständer,1,,Amazon,https://www.amazon.de/KTC-EDLA-Zertifiziert-La...,,859.99,,,,,,
3,4,LL-SJU-2511-0004,St. Julian,innen,EDV & Technik,Drucker,Multifunktionsdrucker,1,,Böttcher AG,https://www.bueromarkt-ag.de/multifunktionsger...,1489.99,1773.09,,,,,,
4,5,LL-SJU-2511-0005,St. Julian,innen,EDV & Technik,3D Drucker,,1,,Bambu Lab Online Shop,https://eu.store.bambulab.com/de/products/h2d?...,,2799.00,,,,,,


# Handling & Migration Products Table

In [ ]:
## Migrate Inventory Table (Schema Mapping)
src_conn = sqlite3.connect(OLD_DB)
tgt_conn = sqlite3.connect(TARGET_DB)

inventory_old = pd.read_sql("SELECT * FROM products;", src_conn)

inventory_new = pd.DataFrame({
    "code": inventory_old["code"],
    "gemeinde": inventory_old["gemeinde"],
    "projekt": inventory_old["projekt"],
    "einsatzort": inventory_old["einsatzort"],
    "kategorie": None, #inventory_old["kategorie"],
    "produkt": None, #inventory_old["produkttyp"],          # renamed
    "produktdetails": inventory_old["produktdetails"],
    "kv_id": None, #inventory_old["inventarnummer"],         # renamed
    "serialnummer": None, #inventory_old["serialnummer"],
    "anzahl": inventory_old["anzahl"],
    "hersteller": inventory_old["hersteller"],
    "preis_netto": inventory_old["preis_netto"],
    "preis_brutto": inventory_old["preis_brutto"],
    "mwst_satz": None, #inventory_old.get("mwst_satz", 19),
    "elo_nummer": None,                               # NEW COLUMN
    "bezahlt": inventory_old["bezahlt"],
    "bestellt_am": inventory_old["bestellt_am"],
    "geliefert_am": inventory_old["geliefert_am"],
    "uebergeben_am": inventory_old["uebergeben_am"],
    "bemerkungen": inventory_old["bemerkungen"],
    "erstellt_von": inventory_old["erstellt_von"],
    "geaendert_von": inventory_old["geaendert_von"],
})

inventory_new.head()


,code,gemeinde,projekt,einsatzort,kategorie,produkt,produktdetails,kv_id,serialnummer,anzahl,...,preis_brutto,mwst_satz,elo_nummer,bezahlt,bestellt_am,geliefert_am,uebergeben_am,bemerkungen,erstellt_von,geaendert_von
0,Adenbach,TEST,,,LAptop,dsd,,,1,,...,Nein,19,None,2026-01-19,2026-01-19,2026-01-19,,cs,cs,cs
1,LL-BEB-2601-0001,Bedesbach,"Museen.Digital, Geoskop",innen,4,Kopfhörer,,TESTsdsdd,ss,1,...,0.0,19,None,Nein,2026-01-27,2026-01-27,2026-01-27,,cs,cs
2,LL-DEI-2601-0001,Deimberg,TESTsdsdd,außen,1,Partybox,,,,1,...,0.0,19,None,Nein,2026-01-27,2026-01-27,2026-01-27,,cs,cs


In [11]:
# Tables from Jana/Lena
# Appending to initial DB (OLD_DB)
st_julian = Path(r'C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\updated_tables\Lena_STJulian.csv')
df_jana = pd.read_csv(st_julian, encoding='utf-8',sep=';')
len(df_jana)


90

In [2]:
# Tables from Jana/Lena
# Appending to initial DB (OLD_DB)
st_julian = Path(r'C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\updated_tables\St.Julian_3.csv')
df_jana = pd.read_csv(st_julian, encoding='utf-8',sep=';')
len(df_jana)

93

In [3]:
df_jana.columns

Index(['ID', 'Code', 'Projekt', 'Anschaffungsjahr', 'Gemeinde', 'Einsatzort',
       'Kategorie', 'Produkt', 'Produktdetails', 'Serialnr. ', 'KV-ID',
       'netto', 'brutto', 'Anzahl', 'ELO-Nummer', 'Geliefert am', 'übergeben',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31'],
      dtype='object')

In [8]:
df_jana.iloc[1]

ID                                                                NaN
Code                                                              NaN
Projekt                                                           NaN
Anschaffungsjahr                                               2025.0
Gemeinde                                                   St. Julian
Einsatzort                                                        DGH
Kategorie                                                         NaN
Produkt                                                       Schrank
Produktdetails          Drehtürenschrank buche 120 cm, Schloss mittig
Serialnr.                                                         NaN
KV-ID                                                             NaN
Einzelpreis / netto                                          351,98 €
Einzelpreis / brutto                                         418,86 €
Anzahl                                                            NaN
ELO-Nummer          

In [9]:
df_jana.head()

,ID,Code,Projekt,Anschaffungsjahr,Gemeinde,Einsatzort,Kategorie,Produkt,Produktdetails,Serialnr.,...,Einzelpreis / brutto,Anzahl,ELO-Nummer,Geliefert am,Lieferumfang,Funktionsprüfung,Notiz,getestet am,getestet von Kürzel,übergeben
0,NaN,NaN,NaN,2025.0,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,...,"418,86 €",NaN,E00103106,15.01.2026,erfolgt im Januar,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,2025.0,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,...,"418,86 €",NaN,E00103106,15.01.2026,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,2025.0,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,...,"418,86 €",NaN,E00103106,15.01.2026,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2025.0,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,...,"418,86 €",NaN,E00103106,15.01.2026,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2025.0,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,...,"418,86 €",NaN,E00103106,15.01.2026,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
## Write Migrated Inventory Data
inventory_new.to_sql("inventory", tgt_conn, if_exists="append", index=False)
print(f"✅ Inventory rows migrated: {len(inventory_new)}")

✅ Inventory rows migrated: 3


In [11]:
## Validation Checks
pd.read_sql("SELECT COUNT(*) AS count FROM inventory;", tgt_conn)


,count
0,3


In [12]:
pd.read_sql("SELECT COUNT(*) AS count FROM users;", tgt_conn)


,count
0,4


In [13]:
inventory_new.isnull().sum()


code              0
gemeinde          0
projekt           0
einsatzort        0
kategorie         0
produkt           0
produktdetails    0
kv_id             0
serialnummer      0
anzahl            0
hersteller        0
preis_netto       0
preis_brutto      0
mwst_satz         0
elo_nummer        3
bezahlt           0
bestellt_am       0
geliefert_am      0
uebergeben_am     0
bemerkungen       0
erstellt_von      0
geaendert_von     0
dtype: int64

# Migrate Users Table

In [6]:
## Create Target Database & Schema
tgt_conn = sqlite3.connect(TARGET_DB)
cur = tgt_conn.cursor()

cur.execute("""
CREATE TABLE inventory (
    code TEXT UNIQUE NOT NULL,
    gemeinde TEXT NOT NULL,
    projekt TEXT,
    einsatzort TEXT,
    kategorie TEXT,
    produkt TEXT,
    produktdetails TEXT,
    kv_id TEXT,
    serialnummer TEXT,
    anzahl INTEGER DEFAULT 1,
    hersteller TEXT,
    preis_netto REAL,
    preis_brutto REAL,
    mwst_satz REAL,
    elo_nummer TEXT,
    bezahlt TEXT,
    bestellt_am TEXT,
    geliefert_am TEXT,
    uebergeben_am TEXT,
    bemerkungen TEXT,
    erstellt_von TEXT,
    geaendert_von TEXT,
    kategorie_typ INTEGER
);
""")

cur.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    role TEXT DEFAULT 'user'
);
""")

tgt_conn.commit()
print("✅ Target schema created")


✅ Target schema created


In [7]:
## Copy `users` Table (Unchanged)
src_conn = sqlite3.connect(NEW_DB)
users_df = pd.read_sql("SELECT * FROM users;", src_conn)
users_df.to_sql("users", tgt_conn, if_exists="append", index=False)

print(f"✅ Users copied: {len(users_df)} rows")

✅ Users copied: 4 rows


## Migration from Jana's Table
TODOs:
- Update the old.db as the main used in the inventory system
- Update code to read new columns or renamed columns
- Implement Bulk importing using the example from Jana CSV
  - Consider issues with UTF-8 reading
  - Code generation must be done in this step
  - make a UI that suggest downloading a Templade and selecting the columns or column mapping from the csv to the corresponding naming of the DB
- Sanity check:
  - Dates and other empty data needed to fill up
  - Duplicated products from old db and new ones provided by Jana

BUGS:
- Delete all function
- Modify table


Further:
- Reports


### 03.02.2026 Table done for 

- TOdO:
  - add Funktionsprüfung (J/N), Notiz if the produkt is 'bad' after prüfung, Getestet am, Getestet vom (Kürsel), Übergeben (J/N)
- Check Kategorie
- Scan option at search bar














# 2026 02 09
- Bezahlt

## Generating labels outside System
### Tables from Jana/Lena



In [37]:
st_julian = Path(r'C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\updated_tables\Lena_STJulian.csv')
df_jana = pd.read_csv(st_julian, encoding='utf-8',sep=';')
print(len(df_jana))
print(df_jana.columns)
print(len(df_jana.columns))

90
Index(['ID', 'Code', 'Projekt', 'Anschaffungsjahr', 'Gemeinde', 'Einsatzort',
       'Kategorie', 'Produkt', 'Produktdetails', 'Serialnr. ', 'KV-ID',
       'Einzelpreis / netto', 'Einzelpreis / brutto', 'Anzahl', 'ELO-Nummer',
       'Geliefert am', 'Lieferumfang', 'Funktionsprüfung', 'Notiz',
       'getestet am', 'getestet von Kürzel ', 'übergeben'],
      dtype='object')
22


In [38]:
# Harmonize naming of headers 
# --------------------------------------------------
# DROP (DELETE) COLUMNS
# --------------------------------------------------
columns_to_drop = [
    "ID",
    "Anschaffungsjahr"
]

df = df_jana.drop(columns=columns_to_drop, errors="ignore")
# errors="ignore" → avoids crash if column does not exist

In [39]:
# --------------------------------------------------
# RENAME COLUMNS
# --------------------------------------------------
rename_map = {
    "Code": "code",
    "Projekt": "projekt",
    "Gemeinde": "gemeinde",
    "Kategorie": "kategorie",
    "Einsatzort": "einsatzort",
    "Produkt": "produkt",
    "Produktdetails": "produktdetails",
    "KV-ID": "kv_id",
    "Serialnr. ": "serialnummer",
    "Einzelpreis / netto": "preis_netto",
    "Einzelpreis / brutto": "preis_brutto",
    "Anzahl": "anzahl",
    "ELO-Nummer": "elo_nummer",
    "Geliefert am": "geliefert_am",
    "Lieferumfang": "lieferumfang",
    "Funktionsprüfung": "funktionspruefung",
    "Notiz": "notiz",
    "getestet am": "getestet_am",
    "getestet von Kürzel ": "getestet_von",
    "übergeben": "uebergeben_am"   
}

df = df.rename(columns=rename_map)

In [40]:
# --------------------------------------------------
# TARGET DATABASE SCHEMA (ORDER MATTERS)
# --------------------------------------------------
db_columns = [
    "code",
    "gemeinde",
    "projekt",
    "einsatzort",
    "kategorie",
    "produkt",
    "produktdetails",
    "kv_id",
    "serialnummer",
    "anzahl",
    "hersteller",
    "preis_netto",
    "preis_brutto",
    "mwst_satz",
    "elo_nummer",
    "bezahlt",
    "bestellt_am",
    "geliefert_am",
    "uebergeben_am",
    "bemerkungen",
    "erstellt_von",
    "geaendert_von",
    "kategorie_typ",
]

# --------------------------------------------------
# SET DEFAULT VALUE FOR BESTELLT_AM
# --------------------------------------------------
df["bestellt_am"] = "2025-11-01"

# --------------------------------------------------
# ADD MISSING COLUMNS
# --------------------------------------------------
for col in db_columns:
    if col not in df.columns:
        df[col] = None

In [41]:
print(len(df))
print(df.columns)
print(len(df.columns))

90
Index(['code', 'projekt', 'gemeinde', 'einsatzort', 'kategorie', 'produkt',
       'produktdetails', 'serialnummer', 'kv_id', 'preis_netto',
       'preis_brutto', 'anzahl', 'elo_nummer', 'geliefert_am', 'lieferumfang',
       'funktionspruefung', 'notiz', 'getestet_am', 'getestet_von',
       'uebergeben_am', 'bestellt_am', 'hersteller', 'mwst_satz', 'bezahlt',
       'bemerkungen', 'erstellt_von', 'geaendert_von', 'kategorie_typ'],
      dtype='object')
28


In [43]:
df.head()

,code,projekt,gemeinde,einsatzort,kategorie,produkt,produktdetails,serialnummer,kv_id,preis_netto,...,getestet_von,uebergeben_am,bestellt_am,hersteller,mwst_satz,bezahlt,bemerkungen,erstellt_von,geaendert_von,kategorie_typ
0,NaN,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,"351,98 €",...,NaN,NaN,2025-11-01,None,None,None,None,None,None,None
1,NaN,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,"351,98 €",...,NaN,NaN,2025-11-01,None,None,None,None,None,None,None
2,NaN,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,"351,98 €",...,NaN,NaN,2025-11-01,None,None,None,None,None,None,None
3,NaN,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,"351,98 €",...,NaN,NaN,2025-11-01,None,None,None,None,None,None,None
4,NaN,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,"351,98 €",...,NaN,NaN,2025-11-01,None,None,None,None,None,None,None


###### Generate codes from DF

In [44]:
import datetime
from collections import defaultdict

def generate_codes_for_df(
    df,
    gemeinde_col="gemeinde",
    date_col="bestellt_am",
    code_col="code",
    gemeinde_abbr_map=None,
    prefix="LL",
):
    """
    Generate LL-ABR-YYMM-#### codes using ONLY the DataFrame.
    Counter resets per Gemeinde + YYMM.
    """

    if gemeinde_abbr_map is None:
        gemeinde_abbr_map = {
            "St. Julian": "SJU",
        }

    counters = defaultdict(int)
    codes = []

    for _, row in df.iterrows():
        gemeinde = row[gemeinde_col]
        purchase_date = row[date_col]

        abbr = gemeinde_abbr_map.get(gemeinde)
        if not abbr:
            raise ValueError(f"No abbreviation defined for Gemeinde: {gemeinde}")

        try:
            dt = datetime.datetime.strptime(purchase_date, "%Y-%m-%d")
        except Exception:
            dt = datetime.date.today()

        yymm = dt.strftime("%y%m")
        key = f"{abbr}-{yymm}"

        counters[key] += 1
        seq = counters[key]

        code = f"{prefix}-{abbr}-{yymm}-{seq:04d}"
        codes.append(code)

    df[code_col] = codes
    return df


In [49]:
df = generate_codes_for_df(df)

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.graphics.barcode import code128
from pathlib import Path
def make_pdf_label(
    code_text,
    output_dir,
    label_text='LandLieben',
    width_mm=22,
    height_mm=6,
):
    """
    Create a single barcode label PDF.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    file_path = output_dir / f"{code_text}.pdf"
    c = canvas.Canvas(str(file_path), pagesize=(width_mm * mm, height_mm * mm))

    # Barcode
    bc = code128.Code128(
        code_text,
        barWidth=0.1 * mm,
        barHeight=height_mm * 0.4 * mm,
    )

    x = (width_mm * mm - bc.width) / 2
    y = (height_mm * mm - bc.height) / 2
    bc.drawOn(c, x, y)

    # Text
    text_to_show = label_text or code_text
    c.setFont("Helvetica", 3)
    text_width = c.stringWidth(text_to_show, "Helvetica", 3)
    c.drawString((width_mm * mm - text_width) / 2, y - 1 * mm, text_to_show)

    c.showPage()
    c.save()

    return file_path


def generate_labels_for_df(
    df,
    code_col="code",
    output_dir="labels",
):
    """
    Generate one PDF label per DataFrame row.
    """
    paths = []

    for code in df[code_col]:
        pdf_path = make_pdf_label(code, output_dir)
        paths.append(pdf_path)

    return paths



In [51]:
output_dir = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\labels_st_julian_03022026")
pdf_paths = generate_labels_for_df(df, output_dir=output_dir)


In [54]:
from PyPDF2 import PdfMerger
def merge_pdfs(
    pdf_paths,
    output_file,
):
    """
    Merge multiple single-page PDFs into one PDF.
    """
    merger = PdfMerger()

    for pdf in pdf_paths:
        merger.append(str(pdf))

    output_file = Path(output_file)
    output_file.parent.mkdir(parents=True, exist_ok=True)

    merger.write(str(output_file))
    merger.close()

    return output_file

final_pdf = merge_pdfs(
    pdf_paths,
    output_file=Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\labels_st_julian_ALL_03022026\all_labels_St_Julian_03022026.pdf"),
)


In [56]:
df.to_csv(Path(r'C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\labels_st_julian_03022026\St_Julian_03022026.csv'))

In [4]:
df1 = pd.read_csv(Path(r'C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\labels_st_julian_ALL_03022026\St_Julian_03022026.csv'))

In [8]:
df1

,Unnamed: 0,code,projekt,gemeinde,einsatzort,kategorie,produkt,produktdetails,serialnummer,kv_id,...,getestet_von,uebergeben_am,bestellt_am,hersteller,mwst_satz,bezahlt,bemerkungen,erstellt_von,geaendert_von,kategorie_typ
0,0,LL-SJU-2511-0001,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,LL-SJU-2511-0002,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,LL-SJU-2511-0003,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,LL-SJU-2511-0004,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,LL-SJU-2511-0005,NaN,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,85,LL-SJU-2511-0086,NaN,St. Julian,DGH,NaN,Schrank,Select Stahlschrank,NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,86,LL-SJU-2511-0087,NaN,St. Julian,DGH,NaN,Schrank,Select Stahlschrank,NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,87,LL-SJU-2511-0088,NaN,St. Julian,DGH,NaN,Schrank,Select Stahlschrank,NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,88,LL-SJU-2511-0089,NaN,St. Julian,DGH,NaN,Schrank,Select Stahlschrank,NaN,NaN,...,NaN,NaN,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df1.columns

Index(['Unnamed: 0', 'code', 'projekt', 'gemeinde', 'einsatzort', 'kategorie',
       'produkt', 'produktdetails', 'serialnummer', 'kv_id', 'preis_netto',
       'preis_brutto', 'anzahl', 'elo_nummer', 'geliefert_am', 'lieferumfang',
       'funktionspruefung', 'notiz', 'getestet_am', 'getestet_von',
       'uebergeben_am', 'bestellt_am', 'hersteller', 'mwst_satz', 'bezahlt',
       'bemerkungen', 'erstellt_von', 'geaendert_von', 'kategorie_typ'],
      dtype='object')

In [7]:
for i in df1.columns:
    print(i)

Unnamed: 0
code
projekt
gemeinde
einsatzort
kategorie
produkt
produktdetails
serialnummer
kv_id
preis_netto
preis_brutto
anzahl
elo_nummer
geliefert_am
lieferumfang
funktionspruefung
notiz
getestet_am
getestet_von
uebergeben_am
bestellt_am
hersteller
mwst_satz
bezahlt
bemerkungen
erstellt_von
geaendert_von
kategorie_typ


## Lena CSV to DB

---- PATH CONFIG ----
- OLD_DB = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\database_handling\inventory_old.db")
- NEW_DB = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\inventory.db")
- TARGET_DB = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\database_handling\harmonized_28012026.db")


In [2]:

#Step 1 — Extract current schema (from existing DB)
import sqlite3
import pandas as pd
from pathlib import Path

old_db_path = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\inventory.db")

conn = sqlite3.connect(old_db_path)
cursor = conn.cursor()

cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
schema_statements = cursor.fetchall()

for stmt in schema_statements:
    print(stmt[0], "\n")

conn.close()


CREATE TABLE sqlite_sequence(name,seq) 

CREATE TABLE options (
            field TEXT NOT NULL,
            value TEXT NOT NULL,
            UNIQUE(field, value)
        ) 

CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        role TEXT DEFAULT 'user'
    ) 

CREATE TABLE inventory (code TEXT, gemeinde TEXT, projekt TEXT, einsatzort TEXT, kategorie TEXT, produkttyp TEXT, produktdetails TEXT, inventarnummer TEXT, serialnummer TEXT, anzahl INTEGER, hersteller TEXT, preis_netto REAL, preis_brutto REAL, bezahlt TEXT, bestellt_am TEXT, geliefert_am TEXT, uebergeben_am TEXT, bemerkungen TEXT, erstellt_von TEXT, geaendert_von TEXT) 

CREATE TABLE options_gemeinde (id INTEGER PRIMARY KEY AUTOINCREMENT, value TEXT UNIQUE) 

CREATE TABLE options_projekt (id INTEGER PRIMARY KEY AUTOINCREMENT, value TEXT UNIQUE) 

CREATE TABLE options_einsatzort (id INTEGER PRIMARY KEY AUTOINCREMENT, value TEXT UNIQU

In [3]:
#Step 2 — Create new database with same schema
new_db_path = Path(r"C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\inventory_new.db")

conn_old = sqlite3.connect(old_db_path)
conn_new = sqlite3.connect(new_db_path)

cursor_old = conn_old.cursor()
cursor_new = conn_new.cursor()

# Exclude internal SQLite tables
cursor_old.execute("""
    SELECT sql FROM sqlite_master 
    WHERE type='table' 
    AND name NOT LIKE 'sqlite_%';
""")

tables = cursor_old.fetchall()

for table_sql in tables:
    if table_sql[0] is not None:
        cursor_new.execute(table_sql[0])

conn_new.commit()
conn_old.close()
conn_new.close()

print("Schema copied successfully (internal tables excluded).")


Schema copied successfully (internal tables excluded).


In [4]:
#Step 3 — Drop ONLY the products table
conn = sqlite3.connect(new_db_path)
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS inventory;")
conn.commit()
conn.close()

print("Old products table removed.")


Old products table removed.


In [5]:
#Step 4 — Create modified products tabl
create_products_sql = """
CREATE TABLE inventory (
    code TEXT UNIQUE NOT NULL,
    projekt TEXT,
    gemeinde TEXT NOT NULL,
    einsatzort TEXT,
    kategorie INTEGER,
    produkt TEXT,
    produktdetails TEXT,
    serialnummer TEXT,
    kv_id TEXT,
    einzelpreis_netto REAL,
    einzelpreis_brutto REAL,
    mwst_satz REAL,
    anzahl INTEGER DEFAULT 1,
    elo_nummer TEXT,
    geliefert_am TEXT,
    lieferumfang TEXT,
    funktionspruefung TEXT,
    notiz TEXT,
    getestet_am TEXT,
    getestet_von TEXT,
    hersteller TEXT,
    anschaffungsjahr TEXT,
    bestellt_am TEXT,
    uebergeben_am TEXT,
    bemerkungen TEXT,
    erstellt_von TEXT,
    geaendert_von TEXT
);
"""

conn = sqlite3.connect(new_db_path)
cursor = conn.cursor()
cursor.execute(create_products_sql)
conn.commit()
conn.close()

print("New products table created.")


New products table created.


In [6]:
df = pd.read_csv(Path(r'C:\Users\crivi\HUB\KV-KUS\code\inventory_web\data\labels_st_julian_ALL_03022026\St_Julian_03022026.csv'))
df.iloc[0]

Unnamed: 0                                                       0
code                                              LL-SJU-2511-0001
projekt                                                        NaN
gemeinde                                                St. Julian
einsatzort                                                     DGH
kategorie                                                      NaN
produkt                                                    Schrank
produktdetails       Drehtürenschrank buche 120 cm, Schloss mittig
serialnummer                                                   NaN
kv_id                                                          NaN
preis_netto                                               351,98 €
preis_brutto                                              418,86 €
anzahl                                                         NaN
elo_nummer                                               E00103106
geliefert_am                                            15.01.

In [7]:
import pandas as pd
import numpy as np

def prepare_products_df_for_db(df):
    df = df.copy()

    # --------------------------------------------------
    # 1️⃣ DROP UNWANTED COLUMNS
    # --------------------------------------------------
    df = df.drop(columns=["Unnamed: 0"], errors="ignore")

    # --------------------------------------------------
    # 2️⃣ RENAME COLUMNS TO MATCH SCHEMA
    # --------------------------------------------------
    rename_map = {
        "preis_netto": "einzelpreis_netto",
        "preis_brutto": "einzelpreis_brutto",
    }

    df = df.rename(columns=rename_map)

    # --------------------------------------------------
    # 3️⃣ CLEAN PRICE COLUMNS (€ → float)
    # --------------------------------------------------
    def clean_price(val):
        if pd.isna(val):
            return None
        val = str(val).replace("€", "").replace(".", "").replace(",", ".").strip()
        try:
            return float(val)
        except:
            return None

    df["einzelpreis_netto"] = df["einzelpreis_netto"].apply(clean_price)
    df["einzelpreis_brutto"] = df["einzelpreis_brutto"].apply(clean_price)

    # --------------------------------------------------
    # 4️⃣ CLEAN DATE FORMAT (DD.MM.YYYY → YYYY-MM-DD)
    # --------------------------------------------------
    def clean_date(val):
        if pd.isna(val):
            return None
        try:
            return pd.to_datetime(val, dayfirst=True).strftime("%Y-%m-%d")
        except:
            return val

    date_columns = ["geliefert_am", "bestellt_am", "uebergeben_am", "getestet_am"]
    for col in date_columns:
        if col in df.columns:
            df[col] = df[col].apply(clean_date)

    # --------------------------------------------------
    # 5️⃣ ADD DEFAULT VALUES
    # --------------------------------------------------
    df["anzahl"] = 1
    df["mwst_satz"] = 19
    df["erstellt_von"] = "cs"
    df["geaendert_von"] = "cs"
    df['projekt'] = "Stationäre Dritte Orte"

    # Anschaffungsjahr from bestellt_am
    if "bestellt_am" in df.columns:
        df["anschaffungsjahr"] = df["bestellt_am"].str[:4]

    # --------------------------------------------------
    # 6️⃣ TARGET SCHEMA (STRICT)
    # --------------------------------------------------
    schema_columns = [
        "code",
        "projekt",
        "gemeinde",
        "einsatzort",
        "kategorie",
        "produkt",
        "produktdetails",
        "serialnummer",
        "kv_id",
        "einzelpreis_netto",
        "einzelpreis_brutto",
        "mwst_satz",
        "anzahl",
        "elo_nummer",
        "geliefert_am",
        "lieferumfang",
        "funktionspruefung",
        "notiz",
        "getestet_am",
        "getestet_von",
        "hersteller",
        "anschaffungsjahr",
        "bestellt_am",
        "uebergeben_am",
        "bemerkungen",
        "erstellt_von",
        "geaendert_von",
    ]

    # --------------------------------------------------
    # 7️⃣ ADD MISSING COLUMNS AS None
    # --------------------------------------------------
    for col in schema_columns:
        if col not in df.columns:
            df[col] = None

    # --------------------------------------------------
    # 8️⃣ KEEP ONLY SCHEMA COLUMNS
    # --------------------------------------------------
    df = df[schema_columns]

    # Replace NaN with None for SQL compatibility
    df = df.where(pd.notnull(df), None)

    return df


In [8]:
df_clean = prepare_products_df_for_db(df)

df_clean.info()
df_clean.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   code                90 non-null     object 
 1   projekt             90 non-null     object 
 2   gemeinde            90 non-null     object 
 3   einsatzort          90 non-null     object 
 4   kategorie           0 non-null      float64
 5   produkt             90 non-null     object 
 6   produktdetails      89 non-null     object 
 7   serialnummer        4 non-null      object 
 8   kv_id               0 non-null      float64
 9   einzelpreis_netto   62 non-null     float64
 10  einzelpreis_brutto  86 non-null     float64
 11  mwst_satz           90 non-null     int64  
 12  anzahl              90 non-null     int64  
 13  elo_nummer          90 non-null     object 
 14  geliefert_am        90 non-null     object 
 15  lieferumfang        5 non-null      object 
 16  funktionsp

,code,projekt,gemeinde,einsatzort,kategorie,produkt,produktdetails,serialnummer,kv_id,einzelpreis_netto,...,notiz,getestet_am,getestet_von,hersteller,anschaffungsjahr,bestellt_am,uebergeben_am,bemerkungen,erstellt_von,geaendert_von
0,LL-SJU-2511-0001,Stationäre Dritte Orte,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",None,NaN,351.98,...,None,None,None,NaN,2025,2025-01-11,None,NaN,cs,cs
1,LL-SJU-2511-0002,Stationäre Dritte Orte,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",None,NaN,351.98,...,None,None,None,NaN,2025,2025-01-11,None,NaN,cs,cs
2,LL-SJU-2511-0003,Stationäre Dritte Orte,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",None,NaN,351.98,...,None,None,None,NaN,2025,2025-01-11,None,NaN,cs,cs
3,LL-SJU-2511-0004,Stationäre Dritte Orte,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",None,NaN,351.98,...,None,None,None,NaN,2025,2025-01-11,None,NaN,cs,cs
4,LL-SJU-2511-0005,Stationäre Dritte Orte,St. Julian,DGH,NaN,Schrank,"Drehtürenschrank buche 120 cm, Schloss mittig",None,NaN,351.98,...,None,None,None,NaN,2025,2025-01-11,None,NaN,cs,cs


In [9]:
df_clean.iloc[0]

code                                               LL-SJU-2511-0001
projekt                                      Stationäre Dritte Orte
gemeinde                                                 St. Julian
einsatzort                                                      DGH
kategorie                                                       NaN
produkt                                                     Schrank
produktdetails        Drehtürenschrank buche 120 cm, Schloss mittig
serialnummer                                                   None
kv_id                                                           NaN
einzelpreis_netto                                            351.98
einzelpreis_brutto                                           418.86
mwst_satz                                                        19
anzahl                                                            1
elo_nummer                                                E00103106
geliefert_am                                    

In [10]:
import sqlite3

conn = sqlite3.connect(new_db_path)
df_clean.to_sql("inventory", conn, if_exists="append", index=False)
conn.close()

print("Data inserted successfully.")


Data inserted successfully.


In [11]:
## Copy data from other tables, e.g Users

conn_old = sqlite3.connect(old_db_path)
conn_new = sqlite3.connect(new_db_path)

cur_old = conn_old.cursor()
cur_new = conn_new.cursor()

# Get all tables except sqlite internal tables and products
cur_old.execute("""
    SELECT name FROM sqlite_master
    WHERE type='table'
    AND name NOT LIKE 'sqlite_%'
    AND name != 'inventory';
""")

tables = cur_old.fetchall()

for (table_name,) in tables:
    print(f"Copying table: {table_name}")

    # Fetch all data from old table
    cur_old.execute(f"SELECT * FROM {table_name}")
    rows = cur_old.fetchall()

    if not rows:
        continue

    # Get column count
    cur_old.execute(f"PRAGMA table_info({table_name})")
    columns = cur_old.fetchall()
    col_count = len(columns)

    placeholders = ",".join(["?"] * col_count)

    cur_new.executemany(
        f"INSERT INTO {table_name} VALUES ({placeholders})",
        rows
    )

conn_new.commit()
conn_old.close()
conn_new.close()

print("All non-products tables copied successfully.")


Copying table: options
Copying table: users
Copying table: options_gemeinde
Copying table: options_projekt
Copying table: options_einsatzort
Copying table: options_kategorie
Copying table: options_produkttyp
Copying table: options_hersteller
All non-products tables copied successfully.
